In [1]:
from Utils_Descriptors import *

c:\Users\ATLAS PRO ELECTRO\AppData\Local\Programs\Python\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
folder = []
for item in os.listdir('data'):
    if(len(item.split('.')) == 1):
        folder.append(item)

In [3]:
folder

['Flore du Cameroun tome 10 1970 Ombellales',
 'Flore du Cameroun tome 12 1972 Loganiaceae',
 'Flore du Cameroun tome 16 1973 Sapindaceae',
 'Flore du Cameroun tome 17 1974 Amaranthaceae',
 'Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae']

# =================================
# Find organes and descriptors

## Construct Dataset for training + Training

Now we need to run this script on all the articles to go throw each paragraph and get all the descriptors ,(the script may need to be stoped manuallay) 

In [4]:
%%time

import random
import tqdm
from spacy.training import Example
import spacy
DATAS = []
paths = ['data/'+ fold for fold in folder]
for path in paths:
    #path = 'data/' + 'Flore du Cameroun tome 12 1972 Loganiaceae'
    pages = especes_pages(path)
    especes, initials = get_especes(pages,path)
    c=0
    sc=0
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.read()
            sentences = lines.replace('\n\n','\n').split('\n')
            k=0
            while(k<len(sentences)):
                sent=sentences[k]

                if(espece_(especes,initials, path,sent) or c==1):
                    #print(espece_(especes,initials, path,sent))
                    sc+=1
                    match=re.search('[a-z\-]*type\s?:|materiel.*camerounais',f_remove_accents(sent))
                    while(not match and k<len(sentences)):
                        sent=sentences[k]
                        sent = sent.replace("\n", "").lower()
                        sent = preprocess(sent)
                        sent = " ".join(word_tokenize(sent))
                        #print('********************')
                        #print(sent)
                        res = descripteur(sent) + organe_(sent) + measure(sent) + color(sent) 
                        #print('**************************')
                        if len(res) != 0:
                            res = {'entities' : res}
                            result = [sent , res]
                            DATAS.append(result)
                        match=re.search('[a-z\-]*type\s?:|materiel.*camerounais',f_remove_accents(sent))
                        if(match):
                            c=0
                        else:
                            c=1
                        k=k+1
                    #print(match)
                k=k+1

e:\M2\NLP\NLP_project-master\Utils_Descriptors.py:281: UserWarning: Skipping measures in the following text because the character span 'de 4 mm , larges de i mm . pétales imbriqués et pubérulents dans' does not align with token boundaries

  warnings.warn(msg)
e:\M2\NLP\NLP_project-master\Utils_Descriptors.py:281: UserWarning: Skipping measures in the following text because the character span 'supérieure de la coupe florale , dorso-médifixes , à filets longs de i mm ,' does not align with token boundaries

  warnings.warn(msg)
e:\M2\NLP\NLP_project-master\Utils_Descriptors.py:281: UserWarning: Skipping measures in the following text because the character span 'glabres ; loges droites , oblongues , longues de i mm ; connectif' does not align with token boundaries

  warnings.warn(msg)
e:\M2\NLP\NLP_project-master\Utils_Descriptors.py:281: UserWarning: Skipping measures in the following text because the character span 'fruit globuleux , glabre , lisse , de i cm de diamètre . graines' doe

Wall time: 7min 43s


In [5]:
#the sc counter is used to calculate the number of paragraphs we extracted data from
print('The number of paragraphs is :'+str(sc))

The number of paragraphs is :123


In [6]:
len(DATAS)

3654

In [7]:
DATAS

[['pl . 4 , 1-3 , p. 19. arbre de i2-i5 m , à frondaison compacte ,',
  {'entities': [[18, 38, 'MEASURE']]}],
 ['parfois épiphyte et finissant par se substituer au tuteur ; rameaux',
  {'entities': [[8, 16, 'DESC']]}],
 ['épais , à entrenœuds courts , portant les bouquets de feuilles et les',
  {'entities': [[54, 62, 'ORGAN']]}],
 ['inflorescences à leur extrémité .', {'entities': [[22, 31, 'FORM']]}],
 ['feuilles pétiolées , digitées , coriaces , glabres : stipules épaisses ,',
  {'entities': [[43, 50, 'DESC'],
    [9, 18, 'FORM'],
    [21, 29, 'FORM'],
    [53, 61, 'ORGAN'],
    [0, 8, 'ORGAN']]}],
 ['intrapétiolaires et largement soudées au pétiole ; pétiole robuste ,',
  {'entities': [[30, 37, 'DESC'], [41, 48, 'ORGAN'], [51, 58, 'ORGAN']]}],
 ['arrondi , strié , atteignant 20-22 cm ; 5-g folioles articulées sur le',
  {'entities': [[53, 63, 'DESC'],
    [10, 15, 'DESC'],
    [44, 52, 'ORGAN'],
    [29, 37, 'MEASURE']]}],
 ['pétiolule de 4-6 cm , limbe elliptique-oblong ( 4,5-8,5 x

We will export this dataset as a spacy file 

In [9]:
len(DATAS[int(len(DATAS)*0.9):])

366

In [13]:
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    skipped_counter=0
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            try:
                span = doc.char_span(start, end, label=label)
            except:
                skipped_counter+=1
                continue

            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
                skipped_counter+=1
            else:
                ents.append(span)
        try:
            doc.ents = ents
            db.add(doc)
        except:
            skipped_counter+=1
            continue
    db.to_disk(output_path)
    return(skipped_counter)

In [17]:

skiped_tr=convert("fr",DATAS[:int(len(DATAS)*0.9)],'ner_data/train_descriptors.spacy')

skiped_ts=convert("fr",DATAS[int(len(DATAS)*0.9):],'ner_data/test_descriptors.spacy')

C:\Users\ATLASP~1\AppData\Local\Temp/ipykernel_20576/769331034.py:23: UserWarning: Skipping entity [53, 57, MEASURE] in the following text because the character span '5 mm' does not align with token boundaries:

'ensemble ; étamines à anthère oblongue , longue de i,5 mm ; disque'

  warnings.warn(msg)
C:\Users\ATLASP~1\AppData\Local\Temp/ipykernel_20576/769331034.py:23: UserWarning: Skipping entity [63, 68, DESC] in the following text because the character span 'lobée' does not align with token boundaries:

'étroit est parfois masqué et le limbe semble pelté , marge 7-9-lobée ,'

  warnings.warn(msg)
C:\Users\ATLASP~1\AppData\Local\Temp/ipykernel_20576/769331034.py:23: UserWarning: Skipping entity [29, 34, DESC] in the following text because the character span 'denté' does not align with token boundaries:

'chaque lobe peu saillant , 3-denté .'

  warnings.warn(msg)
C:\Users\ATLASP~1\AppData\Local\Temp/ipykernel_20576/769331034.py:23: UserWarning: Skipping entity [14, 22, DESC] in the 

In [18]:
print('the number of skiped in train sentences is :',str(skiped_tr))
print('the number of skiped in test sentences is :',str(skiped_ts))

the number of skiped in train sentences is : 499
the number of skiped in test sentences is : 86


In [56]:
import pickle
TD=[]
with open('train_data.pkl','rb') as pick:
    TD.append(pickle.load(pick))

In [57]:
TD=TD[0]

In [58]:
TD[50:220]

[['1 peucedanum winkleri wolff', {'entities': [[2, 21, 'espèce']]}],
 ['2 peucedanum zenkeri engler ex wolff', {'entities': [[2, 20, 'espèce']]}],
 ['3 peucedanum angustisectum ( engler ) norman',
  {'entities': [[2, 26, 'espèce']]}],
 ['1 anthocleista djalonensis a chevalier', {'entities': [[2, 26, 'espèce']]}],
 ["Tessmannia est un genre de plantes dicotylédones de la famille des Fabaceae ( Légumineuses ) , sous-famille des Caesalpinioideae , originaire d'Afrique , qui comprend une douzaine d'espèces acceptées . Ce sont des arbres dont certaines espèces sont exploitées pour leur résine ( copal ) ou leur bois utilisé comme bois d'œuvre ( menuiserie , traverses de chemin de fer , manches d'outils , poteaux ) . 2 anthocleista laxiflora baker",
  {'entities': [[404, 426, 'espèce']]}],
 ['3 anthocleista liebrechtsiana de wild & th dur',
  {'entities': [[2, 29, 'espèce']]}],
 ['4 anthocleista microphylla wernham', {'entities': [[2, 26, 'espèce']]}],
 ['5 anthocleista obanensis wernham', {'

In [59]:
len(TD)

653

creating train vaidation and test sets

In [60]:
DATAC=DATAS.copy()

In [61]:
train=DATAC[0:int(len(DATAC)*0.8)]
test=DATAC[int(len(DATAC)*0.8):len(DATAC)]

now we are going to add species titles to the snetences containing the descreptors 

In [62]:
DATAC=train.copy()

we need to create train and test sets that contains the some kind of corpus that is why we are going to apply the same processing on both 

In [63]:
for i,el in enumerate(TD[0:350]):
    if i in range(100,200):
        #print("########## before:")
        #print(DATAC[i][0])
        #print("######### After:")
        
        DATAC[i][0]=TD[i][0]+'\n'+DATAC[i][0]
        #print(DATAC[i][0])

            #print('######### entites:')
            #print(ent)
            #print('######### old entites:')
            #print(DATAC[i][1]['entities'])
        for j,ent in enumerate(DATAC[i][1]['entities']):
            DATAC[i][1]['entities'][j][0]=DATAC[i][1]['entities'][j][0]+1+len(TD[i][0])
            DATAC[i][1]['entities'][j][1]=DATAC[i][1]['entities'][j][1]+1+len(TD[i][0])
        for j,ent in enumerate(TD[i][1]['entities']):
            DATAC[i][1]['entities'].insert(j,ent)
        #print('##### NEW ENTITIES')
        #print(DATAC[i][1]['entities'])
    else:
        DATAC.append(TD[i])

In [64]:
DATAS[100]

['2 crossonephelis unijugatus ( pellegrin ) leenhouts\nde quelques fleurs . ombelle légère , à pedoncule grêle , sans invo-',
 {'entities': [[2, 27, 'espèce'], [102, 107, 'DESC']]}]

In [65]:
train=DATAC.copy()

In [66]:
train[100]

['2 crossonephelis unijugatus ( pellegrin ) leenhouts\nde quelques fleurs . ombelle légère , à pedoncule grêle , sans invo-',
 {'entities': [[2, 27, 'espèce'], [102, 107, 'DESC']]}]

In [67]:
len(TD)

653

let'us build a test set 

In [68]:
DATAC=test.copy()

In [69]:
len(DATAC)

731

In [70]:

for i in range(50,100):
    #print("########## before:")
    #print(DATAC[i][0])
    #print("######### After:")
    
    DATAC[i][0]=TD[i][0]+'\n'+DATAC[i][0]
    #print(DATAC[i][0])

        #print('######### entites:')
        #print(ent)
        #print('######### old entites:')
        #print(DATAC[i][1]['entities'])
    for j,ent in enumerate(DATAC[i][1]['entities']):
        DATAC[i][1]['entities'][j][0]=DATAC[i][1]['entities'][j][0]+1+len(TD[i][0])
        DATAC[i][1]['entities'][j][1]=DATAC[i][1]['entities'][j][1]+1+len(TD[i][0])
    for j,ent in enumerate(TD[i][1]['entities']):
        DATAC[i][1]['entities'].insert(j,ent)
    #print('##### NEW ENTITIES')
    #print(DATAC[i][1]['entities'])

for el in TD[350:653]:
    DATAC.append(el)

In [73]:
test=DATAC

In [74]:
test[50]

['1 peucedanum winkleri wolff\nde 10-22 cm , pubérulent à glabre , canaliculé .',
 {'entities': [[2, 21, 'espèce'],
   [64, 74, 'DESC'],
   [55, 61, 'DESC'],
   [31, 39, 'MEASURE']]}]

In [77]:

with open ("train_data_desc_mixte.pkl","wb") as pick:
    pickle.dump(train,pick)

In [78]:
import pickle
with open ("test_data_desc_mixte.pkl","r") as pick:
    pickle.dump(test,pick)

In [1]:
import pickle
train=[]
with open("train_data_desc_mixte.pkl",'rb') as pick:
    train.append(pickle.load(pick))
test=[]
with open("test_data_desc_mixte.pkl",'rb') as pick:
    test.append(pickle.load(pick))

In [5]:
train=train[0]
test=test[0]

In [17]:
#trying to copy and run some code
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    skipped_counter=0
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            try:
                span = doc.char_span(start, end, label=label)
            except:
                skipped_counter+=1
                continue

            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
                skipped_counter+=1
            else:
                ents.append(span)
        try:
            doc.ents = ents
            db.add(doc)
        except:
            skipped_counter+=1
            continue
    db.to_disk(output_path)
    return(skipped_counter)

In [18]:
s=convert('fr',train[0:2500],'ner_data/train{:}.spacy'.format(len(train)))
ss=convert('fr',train[2500:3170],'ner_data/valid{:}.spacy'.format(len(train)))

C:\Users\ATLASP~1\AppData\Local\Temp/ipykernel_16344/1447259902.py:24: UserWarning: Skipping entity [22, 37, MEASURE] in the following text because the character span 'ignant 20-22 cm' does not align with token boundaries:

'arrondi , strié , atteignant 20-22 cm ; 5-g folioles articulées sur le'

  warnings.warn(msg)
C:\Users\ATLASP~1\AppData\Local\Temp/ipykernel_16344/1447259902.py:24: UserWarning: Skipping entity [3, 19, MEASURE] in the following text because the character span 'iolule de 4-6 cm' does not align with token boundaries:

'pétiolule de 4-6 cm , limbe elliptique-oblong ( 4,5-8,5 x 12-20 cm ) ,'

  warnings.warn(msg)
C:\Users\ATLASP~1\AppData\Local\Temp/ipykernel_16344/1447259902.py:24: UserWarning: Skipping entity [23, 66, MEASURE] in the following text because the character span 'imbe elliptique-oblong ( 4,5-8,5 x 12-20 cm' does not align with token boundaries:

'pétiolule de 4-6 cm , limbe elliptique-oblong ( 4,5-8,5 x 12-20 cm ) ,'

  warnings.warn(msg)
C:\Users\ATLASP